<a href="https://colab.research.google.com/github/hsalva2/XEMA/blob/main/Reglas%20de%20Asociacion%20Rentabilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 📌 1. Cargar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files  # Para carga manual en Google Colab
from mlxtend.frequent_patterns import apriori, association_rules

# 📌 2. Cargar el archivo CSV manualmente en Google Colab
print("📂 Carga tu archivo CSV con datos y percentiles")
uploaded = files.upload()

# Obtener el nombre del archivo cargado
file_name = list(uploaded.keys())[0]

# 📌 3. Cargar los datos en un DataFrame
df = pd.read_csv(file_name)

# 📌 4. Extraer los percentiles desde el archivo
df_percentiles = df.tail(3).set_index("Anio")  # Últimas 3 filas contienen los percentiles
df = df.iloc[:-3]  # Eliminar las filas de percentiles del dataset original

# 📌 5. Convertir percentiles en un diccionario dinámico (CORREGIDO)
percentiles = {}
for col in df.columns[1:]:  # Omitimos la columna "Anio"
    percentiles[col] = {
        "P25": float(str(df_percentiles.loc["Percentil 25%", col]).replace("%", "").replace(",", "")),
        "Mediana": float(str(df_percentiles.loc["Mediana", col]).replace("%", "").replace(",", "")),
        "P75": float(str(df_percentiles.loc["Percentil 75%", col]).replace("%", "").replace(",", ""))
    }

# 📌 6. Lista de meses y variables a categorizar
meses = ["Febrero", "Abril", "Mayo"]
variables = ["MB%", "GO%", "UN"]

# 📌 7. Función para categorizar todas las variables según percentiles
def categorizar_variables(df, variables, meses):
    df_categorico = df.copy()

    for var in variables:
        for mes in meses:
            col_name = f"{var}_{mes}"
            if col_name in df.columns:
                P25, Mediana, P75 = percentiles[col_name]["P25"], percentiles[col_name]["Mediana"], percentiles[col_name]["P75"]

                etiquetas = [
                    f"{col_name} < {P25}",
                    f"{P25} ≤ {col_name} < {Mediana}",
                    f"{Mediana} ≤ {col_name} < {P75}",
                    f"{col_name} ≥ {P75}"
                ]

                df_categorico[f"{col_name}_Categoria"] = pd.cut(df[col_name].astype(str).str.replace("%", "").str.replace(",", "").astype(float),
                                                                bins=[float("-inf"), P25, Mediana, P75, float("inf")],
                                                                labels=etiquetas,
                                                                include_lowest=True)
                df_categorico.drop(columns=[col_name], inplace=True)  # Eliminar la original

    return df_categorico

# 📌 8. Aplicar la función para categorizar todas las variables
df_categorico = categorizar_variables(df, variables, meses)

# 📌 9. Categorizar `UN_Dic`
P25, Mediana, P75 = percentiles["UN_Dic"]["P25"], percentiles["UN_Dic"]["Mediana"], percentiles["UN_Dic"]["P75"]

etiquetas_un_dic = [
    f"UN_Dic < {P25}",
    f"{P25} ≤ UN_Dic < {Mediana}",
    f"{Mediana} ≤ UN_Dic < {P75}",
    f"UN_Dic ≥ {P75}"
]

df_categorico["UN_Dic"] = pd.cut(df["UN_Dic"].astype(str).str.replace(",", "").astype(float),
                                  bins=[float("-inf"), P25, Mediana, P75, float("inf")],
                                  labels=etiquetas_un_dic,
                                  include_lowest=True)

# 📌 10. Función para generar reglas de asociación SIN min_support y SIN min_confidence
def generar_reglas_asociacion(df, mes):
    print(f"\n📊 Reglas de Asociación - {mes}")

    columnas_mes = [f"MB%_{mes}_Categoria", f"GO%_{mes}_Categoria", f"UN_{mes}_Categoria", "UN_Dic"]
    df_mes = df[columnas_mes].copy()

    # Convertir en formato transaccional (One-Hot Encoding)
    df_encoded = pd.get_dummies(df_mes)

    # 📌 Se usa un soporte mínimo MUY BAJO para evitar errores (pero no 0)
    itemsets_frecuentes = apriori(df_encoded, min_support=0.0001, use_colnames=True)

    # Generar todas las reglas sin filtrar por confianza
    reglas = association_rules(itemsets_frecuentes, metric="lift", min_threshold=1.0)  # Solo reglas con lift > 1

    # Filtrar solo reglas donde UN_Dic sea el consecuente
    reglas = reglas[reglas["consequents"].apply(lambda x: any("UN_Dic" in str(i) for i in x))]

    # Si no hay reglas después del filtrado, advertir al usuario
    if reglas.empty:
        print(f"⚠️ No se encontraron reglas con UN_Dic como consecuente en {mes}.")
        return None

    # Mostrar las reglas principales
    print(reglas[["antecedents", "consequents", "support", "confidence", "lift"]].head(10))

    return reglas

# 📌 11. Generar reglas de asociación para cada mes
reglas_febrero = generar_reglas_asociacion(df_categorico, "Febrero")
reglas_abril = generar_reglas_asociacion(df_categorico, "Abril")
reglas_mayo = generar_reglas_asociacion(df_categorico, "Mayo")


📂 Carga tu archivo CSV con datos y percentiles


Saving Arbol clasificacion.csv to Arbol clasificacion (6).csv

📊 Reglas de Asociación - Febrero
                                          antecedents  \
8          (MB%_Febrero_Categoria_MB%_Febrero < 38.8)   
10         (MB%_Febrero_Categoria_MB%_Febrero < 38.8)   
18  (MB%_Febrero_Categoria_38.8 ≤ MB%_Febrero < 41.9)   
29  (MB%_Febrero_Categoria_41.9 ≤ MB%_Febrero < 43.5)   
31  (MB%_Febrero_Categoria_41.9 ≤ MB%_Febrero < 43.5)   
44         (MB%_Febrero_Categoria_MB%_Febrero ≥ 43.5)   
46         (MB%_Febrero_Categoria_MB%_Febrero ≥ 43.5)   
48         (MB%_Febrero_Categoria_MB%_Febrero ≥ 43.5)   
52         (GO%_Febrero_Categoria_GO%_Febrero < 34.1)   
54         (GO%_Febrero_Categoria_GO%_Febrero < 34.1)   

                          consequents   support  confidence      lift  
8            (UN_Dic_UN_Dic < 2700.8)  0.133333    0.500000  1.875000  
10  (UN_Dic_2700.8 ≤ UN_Dic < 3544.9)  0.133333    0.500000  1.875000  
18           (UN_Dic_UN_Dic ≥ 4296.2)  0.066667    0.250000 

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files  # Para exportar en Google Colab
from mlxtend.frequent_patterns import apriori, association_rules

# 📌 Función para simplificar nombres en los antecedentes y exportar las reglas
def generar_reglas_asociacion(df, mes):
    print(f"\n📊 Reglas de Asociación - {mes}")

    columnas_mes = [f"MB%_{mes}_Categoria", f"GO%_{mes}_Categoria", f"UN_{mes}_Categoria", "UN_Dic"]
    df_mes = df[columnas_mes].copy()

    # Convertir en formato transaccional (One-Hot Encoding)
    df_encoded = pd.get_dummies(df_mes)

    # 📌 Renombrar columnas para evitar nombres duplicados y eliminar "_Categoria_"
    df_encoded.columns = df_encoded.columns.str.replace("_Categoria_", "_").str.replace("UN_Dic_UN_Dic", "UN_Dic")

    # 📌 Se usa un soporte mínimo MUY BAJO para evitar errores (pero no 0)
    itemsets_frecuentes = apriori(df_encoded, min_support=0.0001, use_colnames=True)

    # Generar todas las reglas sin filtrar por confianza
    reglas = association_rules(itemsets_frecuentes, metric="lift", min_threshold=1.0)  # Solo reglas con lift > 1

    # 📌 Filtrar SOLO reglas donde el único consecuente sea UN_Dic
    reglas = reglas[reglas["consequents"].apply(lambda x: len(x) == 1 and any("UN_Dic" in str(i) for i in x))]

    # 📌 Función para limpiar nombres eliminando duplicaciones
    def limpiar_nombres(cadena):
        elementos = sorted(set(cadena.replace("_Categoria_", "_").split(", ")))  # Eliminar duplicados
        elementos = [e.split("_")[-2] + " " + e.split("_")[-1] if "_" in e else e for e in elementos]  # Mantener solo nombre y condición
        return ', '.join(elementos)

    # 📌 Aplicar la limpieza en los antecedentes y consecuentes
    reglas["antecedents"] = reglas["antecedents"].apply(lambda x: limpiar_nombres(str(x)))
    reglas["consequents"] = reglas["consequents"].apply(lambda x: limpiar_nombres(str(x)))

    # 📌 Seleccionar columnas principales para exportar
    reglas_finales = reglas[["antecedents", "consequents", "support", "confidence", "lift"]]

    # 📌 Exportar a Excel
    file_name = f"Reglas_Asociacion_{mes}.xlsx"
    reglas_finales.to_excel(file_name, index=False)
    print(f"✅ Reglas exportadas a {file_name}")

    # 📌 Descargar el archivo automáticamente en Google Colab
    files.download(file_name)

    return reglas_finales

# 📌 Generar y exportar reglas de asociación para cada mes
reglas_febrero = generar_reglas_asociacion(df_categorico, "Febrero")
reglas_abril = generar_reglas_asociacion(df_categorico, "Abril")
reglas_mayo = generar_reglas_asociacion(df_categorico, "Mayo")



📊 Reglas de Asociación - Febrero
✅ Reglas exportadas a Reglas_Asociacion_Febrero.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📊 Reglas de Asociación - Abril
✅ Reglas exportadas a Reglas_Asociacion_Abril.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📊 Reglas de Asociación - Mayo
✅ Reglas exportadas a Reglas_Asociacion_Mayo.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>